<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [16]:
import pandas as pd
import psycopg2
import plotly.express as px
from plotly.subplots import make_subplots

In [17]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [18]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [19]:
query_3_1 = f'''select count(*)
                from vacancies
            '''

In [20]:
vacancies_cnt = pd.read_sql_query(query_3_1, connection).iloc[0][0]
vacancies_cnt

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3358153072.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_cnt = pd.read_sql_query(query_3_1, connection).iloc[0][0]


49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [21]:
query_3_2 = f'''select count(*)
                from employers
            '''

In [22]:
employers_cnt = pd.read_sql_query(query_3_2, connection).iloc[0][0]
employers_cnt

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2709433822.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_cnt = pd.read_sql_query(query_3_2, connection).iloc[0][0]


23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [23]:
query_3_3 = f'''select count(*)
                from areas
            '''

In [24]:
areas_cnt = pd.read_sql_query(query_3_3, connection).iloc[0][0]
areas_cnt

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3053716130.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_cnt = pd.read_sql_query(query_3_3, connection).iloc[0][0]


1362

4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [25]:
query_3_4 = f'''select count(*)
                from industries
            '''

In [26]:
industries_cnt = pd.read_sql_query(query_3_4, connection).iloc[0][0]
industries_cnt

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3542580148.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industries_cnt = pd.read_sql_query(query_3_4, connection).iloc[0][0]


294

***

## Выводы по предварительному анализу данных

In [27]:
# Сформируем таблицу short_info с найденной ранее информацией о числе объектов.
short_info = pd.Series(
    dict(
        (('Вакансии', vacancies_cnt), 
        ('Работодатели', employers_cnt),
        ('Регионы', areas_cnt),
        ('Сферы деятельности', industries_cnt))
    ), name='Информация о количестве'
)

print('Информация о количестве объектов')
short_info

Информация о количестве объектов


Вакансии              49197
Работодатели          23501
Регионы                1362
Сферы деятельности      294
Name: Информация о количестве, dtype: int64

In [70]:
# Поскольку мы пока не понимаем закономерностей и характер распределения величин, построим обобщенную таблицу draft_info, содержащую простые соотношения чисел объектов друг к другу.
# Таким образом, некоторые ячейки этой таблицы будут показывать средние значения (mean) при группировке по соответствующим парамметрам.

draft_info = pd.DataFrame(
    index=short_info.index,
    columns=short_info.index
)

for row in draft_info.index:
    for column in draft_info.columns:
        if row == column: 
            draft_info.loc[row, column] = '-'
        else: 
            draft_info.loc[row, column] = (short_info[row]/short_info[column]).round(2)
        
print('Информация о соотношении числа объектов друг к другу')
draft_info

Информация о соотношении числа объектов друг к другу


,Вакансии,Работодатели,Регионы,Сферы деятельности
Вакансии,-,2.09,36.12,167.34
Работодатели,0.48,-,17.25,79.94
Регионы,0.03,0.06,-,4.63
Сферы деятельности,0.01,0.01,0.22,-


Получается, в среднем:
- на каждый регион приходится 17 работодалей,
- к каждой сфере деятельности относится 80 работодателей,
- каждый работодатель публикует две вакансии,
- и т.п.

Понятно, что эта информация лишь частично отражает действительность, в нюансах помогут разобраться дальнейшие исследования.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [29]:
query_4_1 = f'''select a.name area_name, count(v.id) cnt
                from vacancies v 
                right join areas a on a.id = v.area_id
                group by a.id
                order by cnt desc
            ''' 

In [30]:
vacancies_by_area = pd.read_sql_query(query_4_1, connection)
vacancies_by_area

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3119963648.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_by_area = pd.read_sql_query(query_4_1, connection)


,area_name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
1357,Балакирево,0
1358,Мексика,0
1359,Пугачев,0
1360,Долгоруково,0


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [31]:
query_4_2 = f'''select count(*) cnt
                from vacancies
                where coalesce(salary_from, salary_to) is not null
            '''

In [71]:
with_salary_vacancies = pd.read_sql_query(query_4_2, connection).loc[0, 'cnt']
print(f'Вакансии с полностью или частично заполненной зарплатой: {with_salary_vacancies}.')

# Посчитаем, у какой доли вакансий полностью не указана зарплата.
print(f'Доля вакансий с неуказанной зарплатой: {round(1 - (with_salary_vacancies/vacancies_cnt), 2)}')

Вакансии с полностью или частично заполненной зарплатой: 24073.
Доля вакансий с неуказанной зарплатой: 0.51


C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\4151337690.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [33]:
query_4_3 = f'''select 
                    round(avg(salary_from)) avg_salary_from, 
                    round(avg(salary_to)) avg_salary_to
                from vacancies   
            '''

In [34]:
min_max_salary = pd.read_sql_query(query_4_3, connection)

# Добавим в резуьтат среднее арифметическое.
min_max_salary['mean'] = min_max_salary.loc[0].mean()
min_max_salary

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2211045483.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  min_max_salary = pd.read_sql_query(query_4_3, connection)


,avg_salary_from,avg_salary_to,mean
0,71065.0,110537.0,90801.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [35]:
query_4_4 = f'''select 
                    schedule,
                    employment,
                    count(id) cnt
                from vacancies
                group by schedule, employment
                order by cnt desc
            '''

In [36]:
schedule_employment_cnt = pd.read_sql_query(query_4_4, connection)
schedule_employment_cnt

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\525329997.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schedule_employment_cnt = pd.read_sql_query(query_4_4, connection)


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [37]:
query_4_5 = f'''select 
                    experience,
                    count(*) cnt
                from vacancies
                group by experience
                order by cnt
            '''

In [38]:
pd.read_sql_query(query_4_5, connection)

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3496489661.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_5, connection)


,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы по детальному анализу вакансий

- Количество вакансий распределены по регионам неравномерно. Например, первые топ-5 регионов, к которым относится Москва, Санкт-Петербург и др.крупные города, имеют значительное превышение числа вакансий относительно среднего значения ~36. Здесь очевидна правая ассиметрия распределения.

- 51% вакансий не имеют никаких данных относительно предлагаемой зарплаты, что может затруднить дальнейшую работу с данными, так как этот признак является существенным.

- Данные говорят о том, что средняя минимальная зарплата, которую предлагают работодатели, составаляет прибл.71 тыс.руб., а средняя максимальная - 110 тыс.руб. Таким образом, средняя зарплата равна приблиз.91 тыс.руб.

- Большинство вакансий рассчитаны на полную занятость, они преобладают с большим отрывом. При этом, если рассматривать в разрезе типов трудоустройства, то популярны слудющие комбинации (по убыванию числа вакансий):
а) полная занятость и полный день,
б) частичная занятость и удаленная работа,
в) стажировка и полный день,
г) проектная работа и полный день либо удаленная работа.

- Наиболее популярны среди работодателей вакансии для соискателей с опытом от 1 до 3 лет, а также с опытом от 3 до 6 лет. Вакансий для самых опытных соискателей, наоборот, меньшинство. 
Вероятно, работодатели "нащупывают" золотую середину при поиске будущих работников, нанимая тех, кто уже имеет опыт, чтобы не заниматься обучением, при этом не переплачивая там, где может быть достаточно опыта менее 6 лет.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [39]:
query_5_1 = f'''(select 
                    e.name,
                    count(v.id) cnt
                from employers e
                join vacancies v on v.employer_id = e.id
                group by e.id
                order by cnt desc
                limit 1)

                union all

                (select 
                    e.name,
                    count(v.id) cnt
                from employers e
                join vacancies v on v.employer_id = e.id
                group by e.id
                order by cnt desc
                offset 4 limit 1)
            '''

In [40]:
vacancies_by_employers = pd.read_sql_query(query_5_1, connection)
vacancies_by_employers

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\614488966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_by_employers = pd.read_sql_query(query_5_1, connection)


,name,cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [41]:
query_5_2 = f'''(select
                    a.name, 
                    count(distinct e.id) employers_cnt, 
                    count(distinct v.id) vacancies_cnt
                from areas a 
                left join vacancies v on v.area_id = a.id
                left join employers e on e.area = a.id
                group by a.id
                order by 2 desc, 3 desc
                )

                union all

                (select 
                    a.name, 
                    count(distinct e.id), 
                    count(v.id)
                from areas a
                left join vacancies v on a.id = v.area_id
                join employers e on e.area = a.id
                group by a.id
                having count(v.id) = 0
                order by 2 desc
                limit 1)
            '''


In [42]:
employers_vacancies_by_area = pd.read_sql_query(query_5_2, connection)
employers_vacancies_by_area

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2255315548.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_vacancies_by_area = pd.read_sql_query(query_5_2, connection)


,name,employers_cnt,vacancies_cnt
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698
...,...,...,...
1358,Маргилан,0,0
1359,Термез,0,0
1360,Красноуфимск,0,0
1361,Поварово,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [43]:
query_5_3 = f'''select
                    e.name employer_name,
                    count(distinct v.area_id) areas_cnt
                from employers e
                join vacancies v on v.employer_id = e.id
                join areas a on v.area_id = a.id
                group by e.id
                order by 2 desc
            '''

In [44]:
areas_by_employers = pd.read_sql_query(query_5_3, connection)
areas_by_employers

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2101097458.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_by_employers = pd.read_sql_query(query_5_3, connection)


,employer_name,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [45]:
query_5_4 = f'''select count(e.id) cnt
                from employers e
                left join employers_industries ei on ei.employer_id = e.id
                where ei.industry_id is null
            '''

In [72]:
no_industry_num = pd.read_sql_query(query_5_4, connection).iloc[0][0]

print(f'Число работодателей с неуказанной сферой деятельности: {no_industry_num}')
print(f'Доля таких работодателей: {no_industry_num/employers_cnt:.2f}')

Число работодателей с неуказанной сферой деятельности: 8419
Доля таких работодателей: 0.36


C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2530729582.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [47]:
query_5_5 = f'''select e.name employer_name
                from employers e
                join employers_industries ei on e.id = ei.employer_id
                group by e.id
                having count(ei.industry_id) = 4
                order by 1
                offset 2 limit 1
            '''

In [48]:
pd.read_sql_query(query_5_5, connection)

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3334305160.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_5, connection)


,employer_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [49]:
query_5_6 = f'''select count(e.id) employers_cnt
                from employers e
                join employers_industries ei on e.id = ei.employer_id
                join industries i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
            '''

In [74]:
soft_developers_num = pd.read_sql_query(query_5_6, connection).iloc[0][0]
print('Число разработчиков программного обеспечения:', soft_developers_num)
print(f'Процент разработчиков среди всех вакансий: {round(100*soft_developers_num/employers_cnt)}%')

Число разработчиков программного обеспечения: 3553
Процент разработчиков среди всех вакансий: 15%


C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2555151238.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [51]:
# Код для получения списка городов-милионников.
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
table_rows = page.find('table', class_= 'standard sortable').find_all('tr')[1:]

mln_cities = []
for trow in table_rows: mln_cities.append(trow.find('a').text)

display(mln_cities)


['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [52]:
query_5_7 = f'''(select 
                    a.name,
                    count(v.id) vacancies_cnt
                from vacancies v
                join employers e on e.id = v.employer_id
                join areas a on a.id = v.area_id
                where e.name = 'Яндекс'
                and a.name in {tuple(mln_cities)}
                group by a.id) 
                
                union all
                
                (select 'Total', count(v.id)
                from vacancies v
                join employers e on e.id = v.employer_id
                join areas a on a.id = v.area_id
                where e.name = 'Яндекс'
                and a.name in {tuple(mln_cities)})
                order by 2
            '''
            

In [53]:
mln_cities_yandex = pd.read_sql_query(query_5_7, connection)
print('Распределение вакансий по гордам-миллионникам для компании Яндекс:')
display(mln_cities_yandex)

print('Процент вклада городов-миллионников в число вакансий компании Яндекс:')
print(round(100*mln_cities_yandex[mln_cities_yandex['name'] == 'Total']['vacancies_cnt'].iloc[0]/\
    vacancies_by_employers[vacancies_by_employers['name'] == 'Яндекс'].loc[0, 'cnt']))

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3224257558.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mln_cities_yandex = pd.read_sql_query(query_5_7, connection)


Распределение вакансий по гордам-миллионникам для компании Яндекс:


,name,vacancies_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


Процент вклада городов-миллионников в число вакансий компании Яндекс:
25


***

## Выводы по анализу работодателей

- При распределении вакансий по работодателям первое место занимает компания Яндекс (1933 шт.), число вакансий которой превышает число вакансий компании Газпром нефть(331 шт.), находящейся на пятой строке, в 5.8 раз. Эти значения сильно превышают среднее значение вакансии/регион, равное ~36, что говорит о правой ассиметрии в этом распределении вакансий.

- В каждом регионе может быть разное число работодателей и разное число вакансий, причем соотношение числа вакансий к числу работодателей может разниться у каждого региона. В Москве и Санкт-Петербурге это соотношение колеблется в районе 0.9 - 1.3 вакансии на работодателя. При этом в базе есть регионы, для которых нет ни работодателей, ни вакансий, а также в которых есть работодатели, но нет вакансий (напр., в регионе Россия 410 работодателей, но нет вакансий).

- Каждый работодатель имеет свой набор регионов, где он публикует вакансии. Число регионов может варьироваться от 1 до 181 (лидирует компания Яндекс). Причем упорядоченный по убыванию ряд резко сокаращается: работодатель на пятом месте уже имеет в 2.5 раза меньше регионов относительно максимума.

- 36% работодателей не указали сферу своей деятельности. Будет полезно проанализировать, создаст ли недостающая информация помехи при дальнейшей работе, и дополнить данные.

- Компании-работодатели могут иметь (указывать) разное количество и состав сфер деятельности.

- Среди вакансий 15% разработчиков программного обеспечения.

- Среди вакансий компании Яндекс (лидера по числу вакансий) 25% вакансий приходится на города-миллионники.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [54]:
query_6_1 = f'''select count(*) data_vacancies_cnt
                from vacancies
                where lower(name) like '%data%'
                or lower(name) like '%данн%'
            '''

In [55]:
ds_vacancies_df = pd.read_sql_query(query_6_1, connection)
ds_vacancies_df

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\731843699.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_vacancies_df = pd.read_sql_query(query_6_1, connection)


,data_vacancies_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [56]:
ds_conditions = f'''lower(name) like '%data scientist%'
                    or lower(name) like '%data science%'
                    or lower(name) like '%исследователь данных%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%%'
                    or (name like '%ML%' and name not like '%HTML%')
                '''

junior_conditions = f'''lower(name) like '%junior%'
                        or lower(experience) like '%нет опыта%'
                        or lower(employment) like '%стажировка%'
                    '''

query_6_2 = f'''select count(*)
                from vacancies
                where
                ({ds_conditions})
                and ({junior_conditions})
            '''

In [57]:
junior_ds_vacancies_df = pd.read_sql_query(query_6_2, connection)
junior_ds_vacancies_df

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2773124967.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  junior_ds_vacancies_df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [58]:
key_skills_conditions = f'''lower(key_skills) like '%sql%'
                           or lower(key_skills) like '%postgres%'
                        '''

query_6_3 = f'''select count(*)
                from vacancies
                where
                ({ds_conditions})
                and ({key_skills_conditions})
            '''

In [59]:
sql_postgres_df = pd.read_sql_query(query_6_3, connection)
sql_postgres_df

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3197506625.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_postgres_df = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [60]:
query_6_4 = f'''select count(*)
                from vacancies
                where 
                ({ds_conditions})
                and lower(key_skills) like '%python%'
            '''

In [61]:
python_vacancies_df = pd.read_sql_query(query_6_4, connection)
python_vacancies_df

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\3025189340.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_vacancies_df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [62]:
query_6_5 = f'''select key_skills
                from vacancies
                where
                {ds_conditions}
            '''

In [63]:
key_skills_df = pd.read_sql_query(query_6_5, connection)

key_skills_df['skills_cnt'] = key_skills_df['key_skills'].apply(lambda x: x.count('\t')+1 if x is not None else None)
key_skills_df['skills_cnt'].mean().round(2)

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\2195353813.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills_df = pd.read_sql_query(query_6_5, connection)


6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [64]:
query_6_6 = f'''select  
                    experience, 
                    round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2)) avg_salary
                from vacancies
                where 
                ({ds_conditions})
                and coalesce(salary_from, salary_to) is not null
                group by experience
                having experience = 'От 3 до 6 лет'
            '''

In [65]:
pd.read_sql_query(query_6_6, connection)

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\791858229.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,От 3 до 6 лет,243115.0


***

## Выводы по предметному анализу

- К данным имеет отношение 1771 вакансий, что составляет 3.6% от всего объема вакансий.

- Число вакансий, подходящих для начинающих дата-сайентистов - 51, это 2.88% от DS-вакансий.

- В базе имеется 201 вакансия для DS, в которых в качестве ключевого навыка указан SQL или postgres (или 11.35% от общего числа вакансий)

- Также имеется 351 вакансия, в которых в качестве ключевого навыка указан Python (или 19.82%)

- В вакансии для DS указывают, в среднем, 6.41 ключевых навыка.

- В среднем, дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать на зарплату около 243 тыс.руб./мес., что почти в три раза больше среднего значения зарпалаты 90 тыс. руб., посчитанного ранее для всех вакансий.

In [66]:
# Небольшие дополнительные вычисления

print('Процент вакансий, относящихся к data science:', round(ds_vacancies_df.iloc[0][0]*100/vacancies_cnt, 2))
print('Процент вакансий дата-сайентистов, подходящих для начинающих:', round(junior_ds_vacancies_df.iloc[0][0]*100/ds_vacancies_df.iloc[0][0], 2))
print('Процент data science вакансий, где среди ключевых навыков указаны SQL и postgres:', round(sql_postgres_df.iloc[0][0]*100/ds_vacancies_df.iloc[0][0], 2))
print('Процент data science вакансий, где среди ключевых навыков указан Python:', round(python_vacancies_df.iloc[0][0]*100/ds_vacancies_df.iloc[0][0], 2))

Процент вакансий, относящихся к data science: 3.6
Процент вакансий дата-сайентистов, подходящих для начинающих: 2.88
Процент data science вакансий, где среди ключевых навыков указаны SQL и postgres: 11.35
Процент data science вакансий, где среди ключевых навыков указан Python: 19.82


# Общий вывод по проекту

База данных вакансий содержит существенный объем информации: в ней содержится почти 50 тыс.вакансий от около 23.5 тыс. работодателей, актуальных для более 1.3 тыс регионов. Данные по числу вакансий и работодателей распределены неравномерно. Заметна сильная ассиметрия в пользу отдельных компаний-работодателей и регионов. Что, вероятно, сопряжено с разным экономическим развитием регионов. 
Среди данных имеются пропуски:
- у половины вакансий отсутствуют данные о зарплате. Поскольку эта информация крайне важна для соискателей, нужно искать способы ее дополнения. Например, заменить медианным значением для сферы деятельности и региона. 

- значительные пропуски в принадлежности работодателя к той или иной сфере деятельности, что можеть быть важно для того, чтобы рекомендовать соискателю похожие вакансии. В качестве идеи для заполнения можно предложить использовать моду по признаку Сфера деятельности для вакансий со схожими ключевыми навыками.

Ключевые навыки будут играть значимую роль для рекомендации вакансий, поскольку, если, например, ориентироваться при рекомендации только на компанию-работодателя, интересующего соискателя, то можно по ошибке предложить вакансии из другой сферы деятельности. Признак Ключевые навыки особенно важен для подбора технических вакансий, в том числе интересующих нас DS-вакансий. 

_**Ниже после  выводов по проекту проведем дополнительное исследование, в котором определим, какие именно навыки и как востребованы в вакансиях для дата-сайентистов. Также попытаемся узнать, в каком количестве эти навыки могут быть востребованы компаниями с неуказанной сферой деятельности.**_

Успешность рекомендации вакансии будет зависеть также от таких признаков, как: 

- регион (в том числе, является ли он городом-миллионником), 

- тип трудоустройства и рабочий график,

- опыт работы соискателя,

- желаемая зарплаты (будет иметь значение, какой параметр использьзовать: среднюю зарплату, или верхнюю/нижнюю границу).

Также в качестве дополнительного исследования можно предложить следующее:

- изучить распределение вакансий по сферам деятельности в разрезе регионов,

- в случаях, когда число вакансий мало, изучить возможности расширение спектра подходящих вакансий: например, использовать вакансии для удаленной работы из других регионов, требующие схожие ключевые навыки.

Чтобы улучшить точность рекомендаций было бы интересно учитывать удаленность региона от исходного. А также учитывать готовность соискателя к переезду.


In [76]:
# Дополнительное исследование.
# Узнаем, какие навыки востребованны  в вакансиях, связанных с данными, и как часто они встречаются.

skills_dict = {}

for current_skills in key_skills_df['key_skills']:
    if current_skills is None: continue
    
    skills_lst = current_skills.lower().split('\t')
    
    for elem in skills_lst: 
        if elem in skills_dict: skills_dict[elem] += 1
        else: skills_dict[elem] = 1
    
ds_skills_df = pd.DataFrame(pd.Series(skills_dict).sort_values(ascending=False)).reset_index()
ds_skills_df.columns=['name', 'cnt']
ds_skills_df

# Число вхождений навыка "python" отличается от ранее найденного (351), поскольку в текущем результате разделяются написания.
# Возможно, стоит обдумать способы объединения схожих навыков, чтобы расширить предложение вакансий. 


,name,cnt
0,python,348
1,sql,191
2,machine learning,140
3,git,66
4,data science,64
...,...,...
384,aris,1
385,colvir,1
386,banking compliance,1
387,валютный контроль,1


In [68]:
# Визуализируем на круговой диаграмме популярность ключевых навыков, объединив в отдельную категорию вакансии за пределами топ-7.

top_skills = list(ds_skills_df['name'].iloc[:7])
ds_skills_df['pie_name'] = ds_skills_df['name'].apply(lambda x: x if x in top_skills else 'others')

px.pie(
    ds_skills_df,
    values='cnt',
    names='pie_name',
    title='Популярность навыков в вакансиях для Data Sceince',
    width=700,
    height=450
)

In [69]:
# Из множества вакансий компаний, не указавших сферы деятельности, удалим найденные в п.6.1 DS-вакансии.
# Отсортируем полученные данные по количеству вхождений ключевых навыков для DS.

extra_ds_query = f'''select v.id, v.name, v.key_skills
                    from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                    join vacancies v on v.employer_id = e.id
                    where ei.industry_id is null

                    except 

                    select v.id, v.name, v.key_skills
                    from vacancies v
                    where lower(name) like '%data%'
                    or lower(name) like '%данн%'
                '''
            
extra_ds_df = pd.read_sql_query(extra_ds_query, connection)

skills_list = list(ds_skills_df['name'])

def count_skills(key_skills):
    '''Функция возвращает число DS-навыков, содержащееся в строке key_skills
    '''
    
    num = 0
    
    if key_skills is None: return None
    
    for skill in skills_list: 
        if skill in key_skills: num += 1 
        
    return num
        

extra_ds_df['ds_skills_cnt'] = extra_ds_df['key_skills'].apply(count_skills)

extra_ds_df.sort_values(by='ds_skills_cnt', ascending=False).head(10)

C:\Users\Питерские котики\AppData\Local\Temp\ipykernel_14464\202432598.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,id,name,key_skills,ds_skills_cnt
4664,66104941,Ведущий специалист по защите информации,Настройка сетевых подключений\tНастройка ПК\tИ...,6.0
1409,54932533,Start Teamlead,Python\tJavaScript\tDjango Framework\tReact\tM...,6.0
7717,66176514,Start Teamlead (начинающий),Python\tJavaScript\tDjango Framework\tReact\tM...,6.0
39,55162195,Blockchain Developer / Middle,Git\tLinux\tАнглийский язык\tGolang\tБлокчейн\...,6.0
8678,55139196,Продуктовый аналитик,Маркетинговые исследования\tУправление проекта...,6.0
9550,55537366,Senior Backend Developer NodeJS,Английский язык\tTeamleading\tSQL\tserverless\...,6.0
1397,47990579,PHP / Full-stack Developer,PHP\tjQuery\tJavaScript\tGit\tMySQL\tLaravel\t...,6.0
4720,54561337,Digital маркетолог,Google Analytics\tЯндекс.Метрика\tInternet Mar...,6.0
925,55065070,Lead QA engineer,Функциональное тестирование\tTestRail\tSQL\tQA...,6.0
1007,55081161,Ведущий Senior FullStack JavaScript Developer,Ответственность\tСамостоятельность\tJavaScript...,6.0


Среди отобранных вакансий стали заметны DS-вакансии, не обнаруженные ранее. Таким образом, соискателю можно было бы рекомендовать вакансии с высоким вхождением ключевых навыков.